In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_conf      = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv")
df_death     = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv")
df_recovered = pd.read_csv("../input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv")

print(df_conf.shape)
print(df_death.shape)
print(df_recovered.shape)

In [ ]:
df_conf

In [ ]:
df_conf = df_conf.sort_values(by=['Province/State'])
df_conf = df_conf.reset_index(drop=True)

df_death = df_death.sort_values(by=['Province/State'])
df_death = df_death.reset_index(drop=True)

df_recovered = df_recovered.sort_values(by=['Province/State'])
df_recovered = df_recovered.reset_index(drop=True)

In [ ]:
df_conf

In [ ]:
print(df_conf[["Province/State","Country/Region"]].equals(df_death[["Province/State","Country/Region"]]))

In [ ]:
print(df_conf[["Province/State","Country/Region"]].equals(df_recovered[["Province/State","Country/Region"]]))

In [ ]:
d1= df_conf[['Province/State','Country/Region']]
d2= df_recovered[["Province/State","Country/Region"]]
df_diff = pd.concat([d1,d2]).drop_duplicates(keep=False)
df_diff

In [ ]:
df_recovered[(df_recovered['Country/Region'] == 'Canada')]

In [ ]:
df_conf[(df_conf['Country/Region'] == 'Canada')]

In [ ]:
dates_conf = df_conf.columns[4:]
dates_death = df_death.columns[4:]
print(dates_conf)
print(dates_death)

In [ ]:
dates_conf.equals(dates_death)

In [ ]:
#store dates as values in a new column
df_conf_melted = df_conf.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                              value_vars=dates_conf, 
                              var_name='Date',
                              value_name='Confirmed')
df_conf_melted

In [ ]:
df_death_melted = df_death.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],                                
                                value_vars=dates_death,
                                var_name='Date', 
                                value_name='Deaths')
df_death_melted

In [ ]:
df_combined = pd.concat([df_conf_melted, df_death_melted["Deaths"]], axis = 1, sort = False)
df_combined

**Percentage of deaths**

In [ ]:
df_combined["Percentage"] = df_combined["Deaths"]/df_combined["Confirmed"]
df_combined

In [ ]:
df_combined.fillna(0, inplace=True)
df_combined

In [ ]:
df_combined.dtypes

In [ ]:
import datetime
def format_time(datetime_str):    
    d = datetime.datetime.strptime(datetime_str, '%m/%d/%y')    
    return d.date()

df_combined["Date"] = df_combined["Date"].apply(format_time)
df_combined

**Daily number of confirmed cases and deaths**

In [ ]:
grouped_by_date = df_combined.groupby("Date")

In [ ]:
for date, group in grouped_by_date:
    print("==========")
    print(date)
    print("==========")
    print(group[["Province/State","Country/Region", "Confirmed", "Deaths"]])

**Total daily number of confirmed cases and deaths for each country**

In [ ]:
df_daily = df_combined.groupby(["Date","Country/Region"]).aggregate({'Confirmed': 'sum', 'Deaths': 'sum'})
df_daily

In [ ]:
df_daily.loc[format_time('09/27/20')]

**For a specific day and country**

In [ ]:
df_daily.loc[(format_time('5/22/20'),'US')]

In [ ]:
most_recent_date = df_combined['Date'].max()
print(most_recent_date)

In [ ]:
df_most_recent= df_daily.loc[most_recent_date]
df_most_recent

**Most affected countries**

In [ ]:
df_most_confirmed_recent_sorted = df_most_recent.sort_values(by="Confirmed", ascending=False)
df_most_confirmed_recent_sorted.head(10)

**Countries with the highest numbers of deaths**

In [ ]:
df_most_deaths_recent_sorted = df_most_recent.sort_values(by="Deaths", ascending=False)
df_most_deaths_recent_sorted.head(10)

In [ ]:
df_most_recent.loc["France"]

In [ ]:
df_most_recent[df_most_recent.index == input("Enter country name : ")]

In [ ]:
df_most_recent[df_most_recent.index == "China"]

**The most recent data for China by province**

In [ ]:
df_combined_china = df_combined[df_combined['Country/Region'] == "China"]
df_combined_china

In [ ]:
df_combined_china_date = df_combined_china.groupby(["Date","Province/State"]).aggregate({'Confirmed': 'sum', 'Deaths': 'sum'})
df_combined_china_date

In [ ]:
df_combined_china_most_recent = df_combined_china_date.loc[most_recent_date,:]
df_combined_china_most_recent

In [ ]:
df_combined_china_most_recent_sorted = df_combined_china_most_recent.sort_values(by="Confirmed", ascending=False)
    
df_combined_china_most_recent_sorted.head(10)

In [ ]:
df_recovered_melted = df_recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                              value_vars=dates_conf, 
                              var_name='Date',
                              value_name='Recovered')
df_recovered_melted

In [ ]:
df_recovered_melted["Date"] = df_recovered_melted["Date"].apply(format_time)
df_recovered_melted

In [ ]:
df_recovered_melted.fillna(0, inplace=True)
df_recovered_melted

In [ ]:
df_recovered_daily= df_recovered_melted.groupby(["Date","Country/Region"]).aggregate({'Recovered': 'sum'})
df_recovered_daily

In [ ]:
df_recovered_daily.shape, df_daily.shape

In [ ]:
df_combined_all = pd.concat([df_daily, df_recovered_daily["Recovered"]], axis = 1, sort = False)
df_combined_all

In [ ]:
df_combined_all["Death Percentage"] = (df_combined_all["Deaths"]/df_combined_all["Confirmed"]) *100
df_combined_all["Recovery Percentage"] = (df_combined_all["Recovered"]/df_combined_all["Confirmed"]) *100
df_combined_all

In [ ]:
df_combined_all.fillna(0, inplace=True)
df_combined_all

In [ ]:
df_most_recent= df_combined_all.loc[most_recent_date]
df_most_recent

**Countries with the highest cure rate**

In [ ]:
df_most_recovered_recent_sorted = df_most_recent.sort_values(by="Recovered", ascending=False)
df_most_recovered_recent_sorted.head(10)

# **Visualization** 

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')


for country, df_country in df_combined_all.groupby("Country/Region"):    
    dates = list(df_country.index.get_level_values('Date'))
    confirmed = list(df_country.Confirmed)
    deaths = list(df_country.Deaths)
    recovered= list(df_country.Recovered)
    plt.figure(figsize=(12,7))
    plt.bar(dates, confirmed, color='lightblue', label="Total number of confirmed cases")
    plt.bar(dates, recovered, color='lightgreen',label="Total number of recovered cases")
    plt.bar(dates, deaths, color='red',label="Total number of reported deaths")
    plt.xlabel("Dates")
    plt.ylabel("Number")
    plt.title(country)
    plt.legend()
    plt.show()

In [ ]:
def plot_pie(df, column, title):
    labels = list(df.index)
    sizes = list(df[column])
    explode = [0 for n in range(10)]
    
    with plt.style.context(
        { "axes.prop_cycle" : plt.cycler("color", plt.cm.tab20c.colors)}):
        fig1, ax1 = plt.subplots(figsize=(14,6))
        ax1.pie(sizes, 
                explode=explode, 
                labels=labels, 
                autopct='%1.2f%%', 
                shadow=False, 
                startangle=0)
        ax1.axis('equal')  # Un rapport d'aspect égal garantit que le diagramme sera dessiné en cercle.
        plt.legend()
        plt.title(title)
    
    centre_circle = plt.Circle((0,0),0.70,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    plt.show()

In [ ]:
top10_countries_confirmed = df_most_confirmed_recent_sorted.head(10)
plot_pie(top10_countries_confirmed, "Confirmed", "Top 10 Countries for Confirmed Cases")

In [ ]:
plot_pie(df_combined_china_most_recent_sorted.head(10), "Confirmed", "Top 10 Chinese provinces for confirmed cases")

In [ ]:
plot_pie(df_most_deaths_recent_sorted.head(10), "Deaths", "Top 10 countries for reported deaths")

In [ ]:
plot_pie(df_most_recovered_recent_sorted.head(10), "Recovered", "Top 10 countries for recovered cases")

**Analyze the control of the situation in each country**

In [ ]:
plt.rcParams["figure.figsize"] = (12,7)
for country, df_country in df_combined_all.groupby(level=1):    
    # extract the daily number of confirmed cases and deaths (reverse cumulative sum)
    df_unrolled = df_country.diff().fillna(df_country)
    df_unrolled = df_unrolled.drop(['Death Percentage', 'Recovery Percentage'], axis=1)
    
    plt.plot(dates, df_unrolled["Confirmed"], color='blue', label="Confirmed cases per day")
    plt.plot(dates, df_unrolled["Recovered"], color='green', label="Recovered cases per day")
    plt.plot(dates, df_unrolled["Deaths"], color='red', label="Deaths per day")
    plt.xlabel("Dates")
    plt.ylabel("Number")
    plt.title(country)
    plt.legend()
    plt.show()
    

**Representation on the world map**

In [ ]:
import folium

folium_map = folium.Map(location=[33.886917, 9.537499], zoom_start=2)
color="#E37222"
scale = 200000

In [ ]:
most_recent_date = df_combined['Date'].max()
df_latest_day = df_combined[df_combined.Date==most_recent_date]

for place in df_latest_day.index:
    lat = df_latest_day.loc[place]["Lat"]
    lng = df_latest_day.loc[place]["Long"]    
    confirmed = df_latest_day.loc[place]["Confirmed"]
    deaths = df_latest_day.loc[place]["Deaths"]
   
    # draw a circle on the province / state / region / country
    marker_confirmed = folium.CircleMarker(location=[lat, lng],radius=confirmed/scale,color="blue",fill=True)
    marker_confirmed.add_to(folium_map)
    
    marker_deaths = folium.CircleMarker(location=[lat, lng],radius=deaths/scale,color="red",fill=True)
    marker_deaths.add_to(folium_map)
    
display(folium_map)